# Tratando Valores Ausentes com Imputer
### Inicializando o PySpark

In [3]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Imputer").getOrCreate()

In [4]:
from pyspark.ml.feature import StringIndexer, Imputer

### Carregando a Base de Dados usado nesta Aula

In [7]:
carros = spark.read.load("Material_do_Curso/CarrosNAN.csv", format="csv",
                        sep=";", inferSchema=True, header=True)
carros.show(20)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|       null|             39|2875| null|        0|          1|      4|          4|110|
|    228|        0|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        0|       null|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        0|        360|            315|null| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| null|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

# Aplicando o Imputer

In [10]:
# INstanciando um Objeto, criando o modelo e aplicando a transformação dos dados
imput = Imputer(inputCols=["Cilindradas", "Peso"], outputCols=["nCilindradas", "nPeso"])
modelo = imput.fit(carros)
carros = modelo.transform(carros)

# Mostrando os dados antes de depois de aplica o Imputer
carros.select("Cilindradas", "nCilindradas", "Peso","nPeso").show(20)

+-----------+------------+----+-----+
|Cilindradas|nCilindradas|Peso|nPeso|
+-----------+------------+----+-----+
|        160|         160| 262|  262|
|       null|         848|2875| 2875|
|        108|         108| 232|  232|
|       null|         848|3215| 3215|
|        360|         360|null| 1318|
|        225|         225| 346|  346|
|        360|         360| 357|  357|
|       1467|        1467| 319|  319|
|       1408|        1408| 315|  315|
|       1676|        1676| 344|  344|
|       1676|        1676|null| 1318|
|       2758|        2758| 407|  407|
|       2758|        2758| 373|  373|
|       2758|        2758| 378|  378|
|        472|         472| 525|  525|
|       null|         848|5424| 5424|
|        440|         440|5345| 5345|
|        787|         787|  22|   22|
|        757|         757|1615| 1615|
|        711|         711|1835| 1835|
+-----------+------------+----+-----+
only showing top 20 rows



### Substituindo os valores zero pela mediana na coluna ***Cilindros***

In [12]:
imput = Imputer(inputCols=["Cilindros"], outputCols=["nCilindros"])
imput = imput.setStrategy("median").setMissingValue(0).fit(carros)
imput.transform(carros).select("Cilindros", "nCilindros").show(20)


+---------+----------+
|Cilindros|nCilindros|
+---------+----------+
|        6|         6|
|        6|         6|
|        0|         6|
|        0|         6|
|        0|         6|
|        6|         6|
|        8|         8|
|        4|         4|
|        4|         4|
|        0|         6|
|        6|         6|
|        8|         8|
|        8|         8|
|        8|         8|
|        8|         8|
|        8|         8|
|        8|         8|
|        4|         4|
|        4|         4|
|        4|         4|
+---------+----------+
only showing top 20 rows

